# Auto Insurance Analysis

## Logistic Regression Modeling

## Project Goal

## Summary of Data

### Library Import

In [ ]:
#Import libraries
%run ../python_files/imports

### Data Import

In [ ]:
#Import cleaned, prepared data from EDA
%run ../python_files/auto_insurance_eda

## Modeling

### Logistic Regression Model

logistic regression model for crash vs features

##### Recursive Feature Elimination

In [ ]:
logreg = LogisticRegression()
rfe = RFE(logreg, 46)
rfe = rfe.fit(x_train, y_train.values.ravel())
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
#select columns from RFE

##### Model Implementation

In [ ]:
logit_model=sm.Logit(y_train,x_train)
result=logit_model.fit()
print(result.summary2())

##### Model Fitting

In [ ]:
logreg = LogisticRegression()
logreg.fit(x_train, y_train)

##### Predicting Test Set Results and Calculating Accuracy

In [ ]:
y_pred = logreg.predict(x_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(x_test, y_test)))

##### Confusion Matrix

In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

##### Interpretation of Results

In [ ]:
print(classification_report(y_test, y_pred))

### Linear Regression Model

simple linear regression model for crash_cost vs features

In [ ]:
regressor = LinearRegression()
regressor.fit(x_train, y_train)

#### Model Results

In [ ]:
#Calculate r-squared value

y_pred = regressor.predict(X_test)
print('Linear Regression R squared": %.4f' % regressor.score(X_test, y_test))

In [ ]:
#Calculate root mean squared error (RMSE) value

lin_mse = mean_squared_error(y_pred, y_test)
lin_rmse = np.sqrt(lin_mse)
print('Linear Regression RMSE: %.4f' % lin_rmse)

In [ ]:
#Calculate mean absolute error (MAE) value

lin_mae = mean_absolute_error(y_pred, y_test)
print('Linear Regression MAE: %.4f' % lin_mae)

##### Feature Importance

In [ ]:
feature_labels = np.array(['bathrooms', 'bedrooms', 'finishedsqft', 'totalrooms', 'yearbuilt', 'zindexvalue', 
                           'high_price_high_freq', 'high_price_low_freq', 'low_price', 'Apartment', 'Condominium', 'Cooperative', 
                          'Duplex', 'Miscellaneous', 'Mobile', 'MultiFamily2To4', 'MultiFamily5Plus', 'SingleFamily', 
                           'Townhouse'])
importance = model.feature_importances_
feature_indexes_by_importance = importance.argsort()
for index in feature_indexes_by_importance:
    print('{}-{:.2f}%'.format(feature_labels[index], (importance[index] *100.0)))

## Results and Conclusions

compare crash_percentage vs crash_cost